In [3]:
import os
import folium
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [30]:
# Read data from the xlsx file
grid_info = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "ficheiros_WDL", "DISPOSITIVOS MOVEIS_QUADRICULAS", "DISPOSITIVOS MOVEIS_QUADRICULAS.xlsx")
data = pd.read_excel(grid_info, sheet_name='em bruto', engine='openpyxl')

In [31]:
data.head()

,grelha_id,dicofre,entity_id,entity_type,freguesia,freguesias,grelha_x,grelha_y,latitude,longitude,nome,objectid,position,wkt
0,1,110658,coi.tematica.grelha.lx.atributos.3518,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,4,1,38.692094,-9.225027,Área ribeirinha Pedrouços,1,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.22412591616028 38.691391736...
1,2,110658,coi.tematica.grelha.lx.atributos.3519,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,5,1,38.692094,-9.223231,Área ribeirinha Pedrouços,2,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.22232928559204 38.691391736...
2,3,110658,coi.tematica.grelha.lx.atributos.3520,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,6,1,38.692094,-9.221434,Área ribeirinha Pedrouços,3,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.2205326550238 38.6913917367...
3,4,110658,coi.tematica.grelha.lx.atributos.3521,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,7,1,38.692094,-9.219637,Torre de Belém,4,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.21873602445556 38.691391736...
4,5,110658,coi.tematica.grelha.lx.atributos.3522,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,8,1,38.692094,-9.217841,Torre de Belém,5,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.21693939388732 38.691391736...


In [16]:
# Extract ids
ids = data["grelha_id"].tolist()

In [36]:
# Function to remove extra curly braces and return a valid WKT string
def fix_wkt_string(wkt_string):
    return wkt_string.replace("{{{", "(((").replace("}}}", ")))")

In [40]:
# Remove extra curly braces
data["wkt"] = data["wkt"].apply(fix_wkt_string)

In [41]:
# Apply wkt.loads
polygons = data["wkt"].apply(wkt.loads).tolist()

In [42]:
# Create a GeoDataFrame from the polygons
gdf = gpd.GeoDataFrame({"id": ids, "geometry": polygons})

In [43]:
# Calculate the center of the map (average latitude and longitude)
center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()

In [44]:
# Create a folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

In [45]:
# Add grid squares to the map
for _, row in gdf.iterrows():
    folium.GeoJson(row.geometry, tooltip=f"ID: {row.id}").add_to(m)

In [46]:
# Save the map to an HTML file
m.save("grid_map.html")